In [ ]:
!pip install -qU langchain-community faiss-cpu langchain-google-genai PyMuPDF langchain-groq langgraph

# Vector Database with Faiss

In [31]:
from google.colab import userdata
GEMINI = userdata.get('GEMINI')

In [32]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = GEMINI

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [33]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))) #untuk mendapatkan length vectornya

# initiate Faiss VectorDB
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
# store document to the vector database

from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

## Melakukan pencarian di FaissDB

Operator yang didukung untuk melakukan filtering adalah sebagai berikut:

- $eq (equals)

- $neq (not equals)

- $gt (greater than)

- $lt (less than)

- $gte (greater than or equal)

- $lte (less than or equal)

- $in (membership in list)

- $nin (not in list)

- $and (all conditions must match)

- $or (any condition must match)

- $not (negation of condition)

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": {"$eq": "tweet"}},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
results = vector_store.similarity_search(
    "The new iPhone",
    k=2,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

## Save the DB


In [37]:
vector_store.save_local("faiss_index")

## Load the DB

In [38]:
new_vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

docs = new_vector_store.similarity_search("qux")

In [ ]:
for res in docs:
    print(f"* {res.page_content} [{res.metadata}]")

# RAG dengan LangChain

In [40]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document
import fitz


### Langkah 1: Extract document dan chunk dokumen
Chunk ini dilakukan untuk mengubah dokumen yang panjang menjadi bagian bagian kecil sehingga text yang diterima LLM lebih sedikit. Tujuannya adalah karena adanya batasan token yang diberikan oleh LLM, selain itu semakin panjang text yang diterima oleh LLM semakin lama pula respons yang diberikan oleh LLM.

In [41]:
def extract_text_from_pdf(pdf_path):
    """Mengekstrak teks dari PDF"""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])
    return text

pdf_text = extract_text_from_pdf("/content/Cryptography_Recent_Advances_and_Research_Perspect.pdf")

In [42]:
document = Document(
    page_content=pdf_text
)

In [43]:
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=100,
    chunk_overlap=50
)

chunks = splitter.split_text(pdf_text) #you can also split documents using split_documents

In [ ]:
# hasil chunk document

[chunk[:100] + '...' for chunk in chunks]

### Langkah 2: Upload Chunked Documents ke Database

In [45]:
# inisialisasi faissdb

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))) #untuk mendapatkan length vectornya

# initiate Faiss VectorDB
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [46]:
documents = [Document(page_content=chunk) for chunk in chunks]

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

In [48]:
vector_store.save_local("faiss_cv_index")

### Langkah 3: Develop RAG

In [49]:
# Initialize State

from langchain_core.documents import Document
from typing_extensions import List, TypedDict
from langchain_groq import ChatGroq


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [ ]:
#initialize prompt

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

In [65]:
from google.colab import userdata
GROQ = userdata.get('GROQ')

In [66]:
# Initialize LLM

os.environ["GROQ_API_KEY"] = GROQ

MODEL = 'llama-3.3-70b-versatile'

llm = ChatGroq(
    temperature=0,
    model_name=MODEL
) # kamu bisa mengganti model menjadi gemini atau LLM lainnya

In [60]:
# buat logic untuk mengambil informasi dari faissdb dan men-generate jawaban

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [61]:
# intialize flow
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"question": "What is Crypto?"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

In [ ]:
result = graph.invoke({"question": "What the point of Cryptocurrency?"})
print(f'Answer: {result["answer"]}')